In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths to your dataset
train_path = '/content/drive/MyDrive/train1'
test_path = '/content/drive/MyDrive/test1'

# List few files to confirm
print("Train samples:", os.listdir(train_path)[:5])
print("Test samples:", os.listdir(test_path)[:5])


Mounted at /content/drive
Train samples: ['chicken_031.jpg', 'chicken_063.jpg', 'chicken_002.jpg', 'chicken_048.jpg', 'chicken_054.jpg']
Test samples: ['chicken_102.jpg', 'chicken_103.jpg', 'chicken_101.jpg', 'chicken_120.jpg', 'chicken_118.jpg']


# Define image transform,dataset class and load data


In [2]:
import os
import glob
from PIL import Image
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader


# Use this transform for pretrained models
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class DuckChickenDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = glob.glob(os.path.join(folder_path, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = 0 if 'chicken' in os.path.basename(img_path).lower() else 1
        if self.transform:
            image = self.transform(image)
        return image, label

# Paths from Google Drive
train_path = '/content/drive/MyDrive/train'
test_path  = '/content/drive/MyDrive/test'

# Create dataset and loader
train_dataset = DuckChickenDataset(train_path, transform=transform)
test_dataset  = DuckChickenDataset(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)


# Load and modify a pretrained CNN

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model
model = models.resnet18(pretrained=True)

# Freeze earlier layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for binary classification
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 162MB/s]


# Define loss, optimizer, and finetune the model

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss:.4f}")


Epoch 1 Loss: 8.8927
Epoch 2 Loss: 6.9983
Epoch 3 Loss: 6.2032
Epoch 4 Loss: 5.5868
Epoch 5 Loss: 5.1639


# Evaluation on test set

In [5]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=["Chicken", "Duck"]))


              precision    recall  f1-score   support

     Chicken       0.83      0.75      0.79        20
        Duck       0.77      0.85      0.81        20

    accuracy                           0.80        40
   macro avg       0.80      0.80      0.80        40
weighted avg       0.80      0.80      0.80        40

